## Tables

The tables generated below are used both as guide trees in the progressive
alignment proceedure as well as trees used in the clustering step 
of the regressive alignment proceedure.


The generated tables can be found in the `results/tables/` directory of this repository.



In [ ]:
##########################
# Import python packages #
##########################

import os
import csv

In [ ]:
################################################
# Change directory relative to repository home #
################################################

pwd = os.getcwd()
work_dir=pwd+"/.."
os.chdir(work_dir)
os.getcwd()

In [ ]:
####################################################
# Read in the number of sequences for each dataset #
####################################################

with open("data/num_seqs.csv", mode='r') as infile:
    reader = csv.reader(infile, delimiter='\t')
    sizes_dict = {rows[0]:rows[1] for rows in reader}
    
print(sizes_dict)

In [ ]:
#################################################
# Function to read in accuracy and scores files #
#################################################

def scores_to_dict(scores_dir, scores_dict, tag):
    scores_list=[]
    for score_file in os.listdir(scores_dir):
        family, align_type, bucket, aligner, tree, score_type = score_file.split('.')
        y = tuple([tag, align_type, aligner, tree, family, score_type])
        with open(scores_dir + score_file, 'r') as infile:
            data = infile.read()
        scores_dict[y]=data.rstrip()
    return scores_dict

In [ ]:
##################################################
# Read in the full and reference datasets scores #
##################################################

scores_dict = {}
full_scores_dir="results/individual_scores/"
scores_dict = scores_to_dict(full_scores_dir, scores_dict, "full")

ref_scores_dir="results_reference/individual_scores/"
scores_dict = scores_to_dict(ref_scores_dir, scores_dict, "ref")



In [ ]:
#################################################################################
# Create a dictionary for average TC scores and CPU time when above 10,000 seqs #
#################################################################################

# Dictionary of the average TC scores and CPU time
full_top20_tc_avg_dict={}
ref_top20_tc_avg_dict={}
cpu_top20_dict={}

# Take all datasets and create a dictionary of average scores
datasets=set([k[1]+'/'+k[2]+'/'+k[3] for k,v in scores_dict.items() ])

# Calculate average TC score for each of the full datasets with over 10,000 sequences
for dataset in datasets:
    key = dataset.split("/")
    i = [v for k,v in scores_dict.items() if k[0]=='full' and k[1]==key[0] and k[2]==key[1] and k[3]==key[2] and int(sizes_dict[k[4]]) > 10000 and k[5]=='tc']
    l = [float(j) for j in i]
    if (len(l)==20):
        avg = sum(l)/float(len(l))
        full_top20_tc_avg_dict[tuple(key)]=avg

# Calculate average TC for each of the reference datasets
for dataset in datasets:
    key = dataset.split("/")
    i = [v for k,v in scores_dict.items() if k[0]=='ref' and k[1]==key[0] and k[2]==key[1] and k[3]==key[2] and int(sizes_dict[k[4]]) > 10000 and k[5]=='tc']
    l = [float(j) for j in i]
    if (len(l)==20):
        avg = sum(l)/float(len(l))
        ref_top20_tc_avg_dict[tuple(key)]=avg

# Calculate average CPU time required for each the datasets with over 10,000 sequences
for dataset in datasets:
    key = dataset.split("/")
    i = [v for k,v in scores_dict.items() if k[0]=='full' and k[1]==key[0] and k[2]==key[1] and k[3]==key[2] and int(sizes_dict[k[4]]) > 10000 and k[5]=='cpu']
    l = [float(j) for j in i]
    if (len(l)==20):
        avg = sum(l)/float(len(l))
        cpu_top20_dict[tuple(key)]=avg

In [ ]:
#########################
# Export Table 1 as CSV #
#########################

with open('results/tables/table1.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',')

    filewriter.writerow(['','','non-regressive','regressive','reference','non-regressive','regressive'])
    filewriter.writerow(['tree method','MSA algorithm','score %','score %','score %','cpu time (ms)','cpu time (ms)'])

    f_dic, r_dic, cpu_dic, aligner, tree = full_top20_tc_avg_dict, ref_top20_tc_avg_dict, cpu_top20_dict, 'MAFFT-FFTNS1', 'MAFFT_PARTTREE'
    filewriter.writerow(['PartTree','Fftns1',round(f_dic['std_align',aligner,tree],2),round(f_dic['dpa_align',aligner,tree],2),round(r_dic['std_align',aligner,tree],2),cpu_dic['std_align',aligner,tree],cpu_dic['dpa_align',aligner,tree]])
 
    f_dic, r_dic, cpu_dic, aligner, tree = full_top20_tc_avg_dict, ref_top20_tc_avg_dict, cpu_top20_dict, 'MAFFT-FFTNS1', 'CLUSTALO'
    filewriter.writerow(['mBed','Fftns1',round(f_dic['std_align',aligner,tree],2),round(f_dic['dpa_align',aligner,tree],2),round(r_dic['std_align',aligner,tree],2),cpu_dic['std_align',aligner,tree],cpu_dic['dpa_align',aligner,tree]])
 
    f_dic, r_dic, cpu_dic, aligner, tree = full_top20_tc_avg_dict, ref_top20_tc_avg_dict, cpu_top20_dict, 'CLUSTALO', 'MAFFT_PARTTREE'
    filewriter.writerow(['PartTree','ClustalO',round(f_dic['std_align',aligner,tree],2),round(f_dic['dpa_align',aligner,tree],2),round(r_dic['std_align',aligner,tree],2),cpu_dic['std_align',aligner,tree],cpu_dic['dpa_align',aligner,tree]])

    f_dic, r_dic, cpu_dic, aligner, tree = full_top20_tc_avg_dict, ref_top20_tc_avg_dict, cpu_top20_dict, 'CLUSTALO', 'CLUSTALO'
    filewriter.writerow(['mBed','ClustalO',round(f_dic['std_align',aligner,tree],2),round(f_dic['dpa_align',aligner,tree],2),round(r_dic['std_align',aligner,tree],2),cpu_dic['std_align',aligner,tree],cpu_dic['dpa_align',aligner,tree]])
    
    filewriter.writerow(['Average','','','','','',''])
    filewriter.writerow(['','','','','','',''])
    
    f_dic, r_dic, cpu_dic, aligner, tree = full_top20_tc_avg_dict, ref_top20_tc_avg_dict, cpu_top20_dict, 'UPP', 'DEFAULT'
    filewriter.writerow(['default/mBed','UPP',round(f_dic['default_align',aligner,tree],2),round(f_dic['dpa_align',aligner,'CLUSTALO'],2),round(r_dic['default_align',aligner,tree],2),cpu_dic['default_align',aligner,'DEFAULT'],cpu_dic['dpa_align',aligner,'CLUSTALO']])
 
    f_dic, r_dic, cpu_dic, aligner, tree = full_top20_tc_avg_dict, ref_top20_tc_avg_dict, cpu_top20_dict, 'MAFFT-SPARSECORE', 'DEFAULT'
    filewriter.writerow(['default/mBed','Sparsecore',round(f_dic['default_align',aligner,tree],2),round(f_dic['dpa_align',aligner,'CLUSTALO'],2),round(r_dic['default_align',aligner,tree],2),cpu_dic['default_align',aligner,'DEFAULT'],cpu_dic['dpa_align',aligner,'CLUSTALO']])
 
    f_dic, r_dic, cpu_dic, aligner, tree = full_top20_tc_avg_dict, ref_top20_tc_avg_dict, cpu_top20_dict, 'MAFFT-GINSI', 'MAFFT_PARTTREE'
    filewriter.writerow(['PartTree','Gins1','-',round(f_dic['dpa_align',aligner,tree],2),round(r_dic['std_align',aligner,tree],2),'-',cpu_dic['dpa_align',aligner,tree]])

    f_dic, r_dic, cpu_dic, aligner, tree = full_top20_tc_avg_dict, ref_top20_tc_avg_dict, cpu_top20_dict, 'MAFFT-GINSI', 'CLUSTALO'
    filewriter.writerow(['mBed','Gins1','-',round(f_dic['dpa_align',aligner,tree],2),round(r_dic['std_align',aligner,tree],2),'-',cpu_dic['dpa_align',aligner,tree]]) 


In [ ]:
#######################################
# Export all raw results as CSV table #
#######################################

with open('results/tables/full_table_sp.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',')
    for e in scores_dict:
        if e[5] == 'sp':
            filewriter.writerow([e[0], e[1], e[2], e[3], e[4], sizes_dict[e[4]], scores_dict[e]])
            
with open('results/tables/full_table_tc.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',')
    for e in scores_dict:
        if e[5] == 'tc':
            filewriter.writerow([e[0], e[1], e[2], e[3], e[4], sizes_dict[e[4]], scores_dict[e]])

with open('results/tables/full_table_cpu.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',')
    for e in scores_dict:
        if e[5] == 'cpu':
            filewriter.writerow([e[0], e[1], e[2], e[3], e[4], sizes_dict[e[4]], scores_dict[e]])

In [ ]:
##################################
# Export table for CCA Figure 2B #
##################################

with open('results/tables/figure2b.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',')
    
    f_dic = full_top20_tc_avg_dict
    
    filewriter.writerow(['reg_method','tree_method','align_method','accuracy'])
    
    filewriter.writerow(['Non-Regressive','PartTree','Fftns1',f_dic['std_align','MAFFT-FFTNS1','MAFFT_PARTTREE']])
    filewriter.writerow(['Non-Regressive','mBed','Fftns1',f_dic['std_align','MAFFT-FFTNS1','CLUSTALO']])
    filewriter.writerow(['Non-Regressive','PartTree','ClustalO',f_dic['std_align','CLUSTALO','MAFFT_PARTTREE']])
    filewriter.writerow(['Non-Regressive','mBed','ClustalO',f_dic['std_align','CLUSTALO','CLUSTALO']])
    
    filewriter.writerow(['Non-Regressive','UPP Tree','UPP',f_dic['default_align','UPP','DEFAULT']])
    filewriter.writerow(['Non-Regressive','Sparsecore Tree','Sparsecore',f_dic['default_align','MAFFT-SPARSECORE','DEFAULT']])
    
    filewriter.writerow(['Regressive','PartTree','Fftns1',f_dic['dpa_align','MAFFT-FFTNS1','MAFFT_PARTTREE']])
    filewriter.writerow(['Regressive','mBed','Fftns1',f_dic['dpa_align','MAFFT-FFTNS1','CLUSTALO']])
    filewriter.writerow(['Regressive','PartTree','ClustalO',f_dic['dpa_align','CLUSTALO','MAFFT_PARTTREE']])
    filewriter.writerow(['Regressive','mBed','ClustalO',f_dic['dpa_align','CLUSTALO','CLUSTALO']])
    
    filewriter.writerow(['Regressive','mBed','UPP',f_dic['dpa_align','UPP','CLUSTALO']])

    filewriter.writerow(['Regressive','mBed','Sparsecore',f_dic['dpa_align','MAFFT-SPARSECORE','CLUSTALO']])
    filewriter.writerow(['Regressive','PartTree','Sparsecore',f_dic['dpa_align','MAFFT-SPARSECORE','MAFFT_PARTTREE']])
    
    filewriter.writerow(['Regressive','PartTree','Gins1',f_dic['dpa_align','MAFFT-GINSI','MAFFT_PARTTREE']])
    filewriter.writerow(['Regressive','mBed','Gins1',f_dic['dpa_align','MAFFT-GINSI','CLUSTALO']])


In [ ]:
##############################
# Export table for Figure 3A #
##############################
with open('results/tables/figure3b.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',')
    
    f_dic = full_top20_tc_avg_dict
    cpu_dic = cpu_top20_dict

    filewriter.writerow(['method','aggregation','score','cpu'])

    filewriter.writerow(['Fftns1-PartTree','non-regressive',f_dic['std_align','MAFFT-FFTNS1','MAFFT_PARTTREE'],cpu_dic['std_align','MAFFT-FFTNS1','MAFFT_PARTTREE']])
    filewriter.writerow(['Fftns1-PartTree','regressive',f_dic['dpa_align','MAFFT-FFTNS1','MAFFT_PARTTREE'],cpu_dic['dpa_align','MAFFT-FFTNS1','MAFFT_PARTTREE']]) 

    filewriter.writerow(['Fftns1-mBed','non-regressive',f_dic['std_align','MAFFT-FFTNS1','CLUSTALO'],cpu_dic['std_align','MAFFT-FFTNS1','CLUSTALO']])
    filewriter.writerow(['Fftns1-mBed','regressive',f_dic['dpa_align','MAFFT-FFTNS1','CLUSTALO'],cpu_dic['dpa_align','MAFFT-FFTNS1','CLUSTALO']])
    
    filewriter.writerow(['ClustalO-PartTree','non-regressive',f_dic['std_align','CLUSTALO','MAFFT_PARTTREE'],cpu_dic['std_align','CLUSTALO','MAFFT_PARTTREE']])
    filewriter.writerow(['ClustalO-PartTree','regressive',f_dic['dpa_align','CLUSTALO','MAFFT_PARTTREE'],cpu_dic['dpa_align','CLUSTALO','MAFFT_PARTTREE']])

    filewriter.writerow(['ClustalO-mBed','non-regressive',f_dic['std_align','CLUSTALO','CLUSTALO'],cpu_dic['std_align','CLUSTALO','CLUSTALO']])
    filewriter.writerow(['ClustalO-mBed','regressive',f_dic['dpa_align','CLUSTALO','CLUSTALO'],cpu_dic['dpa_align','CLUSTALO','CLUSTALO']])

    filewriter.writerow(['UPP-mBed','non-regressive',f_dic['default_align', 'UPP', 'DEFAULT'],cpu_dic['default_align', 'UPP', 'DEFAULT']])
    filewriter.writerow(['UPP-mBed','regressive',f_dic['dpa_align','UPP', 'CLUSTALO'],cpu_dic['dpa_align', 'UPP', 'CLUSTALO']])

    filewriter.writerow(['Sparsecore-mBed','non-regressive',f_dic['default_align', 'MAFFT-SPARSECORE', 'DEFAULT'],cpu_dic['default_align', 'MAFFT-SPARSECORE', 'DEFAULT']])
    filewriter.writerow(['Sparsecore-mBed','regressive',f_dic['dpa_align','MAFFT-SPARSECORE', 'CLUSTALO'],cpu_dic['dpa_align', 'MAFFT-SPARSECORE', 'CLUSTALO']])
 
    filewriter.writerow(['Ginsi-mBed','regressive',f_dic['dpa_align', 'MAFFT-GINSI', 'CLUSTALO'],cpu_dic['dpa_align', 'MAFFT-GINSI', 'CLUSTALO']])
    filewriter.writerow(['Ginsi-PartTree','regressive',f_dic['dpa_align', 'MAFFT-GINSI', 'MAFFT_PARTTREE'],cpu_dic['dpa_align', 'MAFFT-GINSI', 'MAFFT_PARTTREE']])
